<a href="https://colab.research.google.com/github/cecs10/DIO-trabalhos/blob/main/NovoNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q openai
!apt-get install -y ffmpeg


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 10.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [2]:
# ================================
# CONFIGURAÇÃO
# ================================
language = 'pt'
audio_seconds = 6

# ================================
# GRAVAÇÃO DE ÁUDIO
# ================================
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  const recorder = new MediaRecorder(stream)
  const chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    const blob = new Blob(chunks)
    const text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec):
  display(Javascript(RECORD))
  js_result = output.eval_js(f'record({sec * 1000})')
  audio = b64decode(js_result.split(',')[1])
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  return f'/content/{file_name}'

print("🎧 Ouvindo... autorize o microfone quando aparecer")
record_file = record(audio_seconds)
display(Audio(record_file, autoplay=False))


# ================================
# WHISPER SMALL
# ================================
import whisper

model = whisper.load_model("small")
result = model.transcribe(
    record_file,
    fp16=False,
    language=language
)

transcription = result["text"]
print("\n📝 Transcrição:")
print(transcription)


# ================================
# OPENAI (API NOVA, CHAVE OCULTA)
# ================================
import os
from google.colab import userdata
from openai import OpenAI

# 🔐 LÊ O SECRET DO COLAB E JOGA PARA O ENV
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# agora sim o cliente enxerga a chave
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1-mini",
    input=transcription
)

print("\n🤖 Resposta do modelo:")
print(response.output_text)
  # usa automaticamente o Secret OPENAI_API_KEY

response = client.responses.create(
    model="gpt-4.1-mini",
    input=transcription
)

print("\n🤖 Resposta do modelo:")
print(response.output_text)


🎧 Ouvindo... autorize o microfone quando aparecer


<IPython.core.display.Javascript object>


📝 Transcrição:
 Quem criou o Google?


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}